In [1]:
from pyautocad import Autocad, APoint
import geopandas as gpd
from shapely.geometry import LineString

def add_shapefile_layer_and_offset(dwg_file_path, shapefile_path, offset_distance, output_file_path):
    # Initialize AutoCAD
    acad = Autocad(create_if_not_exists=True)
    acad.prompt("AutoCAD script started\n")
    print("Initialized AutoCAD")

    # Open the DWG file
    acad.app.Documents.Open(dwg_file_path)
    doc = acad.app.ActiveDocument
    modelspace = doc.ModelSpace
    print(f"Opened DWG file: {dwg_file_path}")

    # Load the shapefile
    gdf = gpd.read_file(shapefile_path)
    print(f"Loaded shapefile: {shapefile_path}")

    # Add the original layer with specific attributes if it doesn't exist
    layer_name = '_scritte_sottile'
    try:
        layer = doc.Layers.Item(layer_name)
        print(f"Layer '{layer_name}' already exists")
    except:
        layer = doc.Layers.Add(layer_name)
        layer.Color = 255
        layer.Linetype = 'trattopuntox2'
        layer.Lineweight = 0.05
        print(f"Created new layer: {layer_name}")

    # Add lines from the shapefile to the DWG
    for geom in gdf.geometry:
        if isinstance(geom, LineString):
            coords = list(geom.coords)
            points = [APoint(x, y) for x, y in coords]
            polyline = modelspace.AddPolyline(points)
            polyline.Layer = layer_name
            print(f"Added polyline to layer '{layer_name}'")

    # Create offsets of the lines in the new layer
    for entity in modelspace:
        if entity.Layer == layer_name and entity.ObjectName == 'AcDbPolyline':
            offset_entities = entity.Offset(offset_distance)
            print(f"Created offsets for entity in layer '{layer_name}'")

            # Add new layers for the offsets with specific attributes if they don't exist
            offset_layer = '_tracciolino'
            try:
                offset_layer_obj = doc.Layers.Item(offset_layer)
                print(f"Offset layer '{offset_layer}' already exists")
            except:
                offset_layer_obj = doc.Layers.Add(offset_layer)
                offset_layer_obj.Color = 30
                offset_layer_obj.Linetype = 'Continuous'
                offset_layer_obj.Lineweight = 0.09
                print(f"Created new offset layer: {offset_layer}")

            # Add offset lines to the new layers
            for offset_entity in offset_entities:
                offset_entity.Layer = offset_layer
                print(f"Added offset entity to layer '{offset_layer}'")

    # Save the modified DWG file
    doc.SaveAs(dwg_file_path)
    print(f"Saved modified DWG file: {dwg_file_path}")

dwg_file_path = r'S:\progetti\1173_STRADE VICINALI MODIGLIANA_Ord. 13-23_001731_Varianti TREBBIO\1173_Cartografia\1173_curve_di_livello\1173_FUCA\1173_FUCA.dwg'
shapefile_path = r'S:\progetti\1173_STRADE VICINALI MODIGLIANA_Ord. 13-23_001731_Varianti TREBBIO\1173_Cartografia\1173_curve_di_livello\1173_FUCA\1173_FUCA.shp'
offset_distance = 1.5
output_file_path = r'S:\progetti\1173_STRADE VICINALI MODIGLIANA_Ord. 13-23_001731_Varianti TREBBIO\1173_Cartografia\1173_curve_di_livello\1173_FUCA\1173_FUCA_planimetria.dwg'

add_shapefile_layer_and_offset(dwg_file_path, shapefile_path, offset_distance, output_file_path)

print(f"Shapefile '{shapefile_path}' added as layer '_scritte_sottile' and offset by {offset_distance} meters in both directions for file '{dwg_file_path}'.")

AutoCAD script started



OSError: [WinError -2147221005] Stringa dell'interfaccia non valida